In [1]:
import pandas as pd
import numpy as np

In [2]:
data_demo = pd.read_csv("data/demographics_1.csv")

In [3]:
data_demo.head()

,PostCode,Ethnicity,Religion,Language,Immigration,Commute,Marital Status,Family Size,Dwelling,Ownership,Education,Profession,Income,Family Income,Age,Gender
0,E1V6V5,North American,Christian,English,Non-immigrants,"Car, truck, van - as a driver",Married,1 person,Single-detached house,Owned,Postsecondary,62 Health care and social assistance,34560,150658.0,25 to 29 years,Female
1,E1V6V5,British,Christian,French,Non-immigrants,"Car, truck, van - as a driver",Married,2 persons,Single-detached house,Owned,Postsecondary,"56 Administrative and support, waste managemen...",57785,78950.0,45 to 49 years,Male
2,A0A0B7,British,Christian,English and non-official language,Non-immigrants,"Car, truck, van - as a driver",Married,1 person,Single-detached house,Owned,College,31-33 Manufacturing,44055,111262.0,30 to 34 years,Female
3,A0A0B7,North American,Christian,English and non-official language,Non-immigrants,"Car, truck, van - as a driver",Married,2 persons,Single-detached house,Owned,Postsecondary,44-45 Retail trade,35954,64925.0,35 to 39 years,Male
4,E4T0C1,North American,Christian,French,Non-immigrants,"Car, truck, van - as a driver",Married,2 persons,Single-detached house,Owned,Secondary school,"21 Mining, quarrying, and oil and gas extraction",59829,154926.0,60 to 64 years,Female


In [4]:
data_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29115181 entries, 0 to 29115180
Data columns (total 16 columns):
PostCode          object
Ethnicity         object
Religion          object
Language          object
Immigration       object
Commute           object
Marital Status    object
Family Size       object
Dwelling          object
Ownership         object
Education         object
Profession        object
Income            int64
Family Income     float64
Age               object
Gender            object
dtypes: float64(1), int64(1), object(14)
memory usage: 3.5+ GB


## Select features that are potentially correlated with the feature (number of trades) from the aggregated data, and are easily aggregated as well (nominal features that only have two categories).
The candadites are:
1. Immigration -- nominal
2. Gender -- nominal
3. Ownership -- nominal
4. Dwelling -- nominal
5. Education -- nominal
6. ethnicity -- nominal
7. Commute -- nominal


6. Age -- ordinal


7. Income -- numeric
8. Family Size -- numeric
9. Family Income -- numeric

In [5]:
removed_col = ["Religion", "Language", "Marital Status", "Profession"]
data_demo.drop(removed_col, inplace=True, axis=1)

## Aggregate norminal data

In [9]:
def aggregate_group(x):
    d = {}
    d["Immigrants Percentage"] = x["Immigration"].value_counts()["Immigrants"] / x["Immigration"].count()\
        if "Immigrants" in x["Immigration"].value_counts() else 0
    d["Owner Percentage"] = x["Ownership"].value_counts()["Owned"] / x["Ownership"].count()\
        if "Owned" in x["Ownership"].value_counts() else 0
    d["House Percentage"] = x["Dwelling"].value_counts()["Single-detached house"] / x["Dwelling"].count()\
        if "Single-detached house" in x["Dwelling"].value_counts() else 0
    d["Male Percentage"] = x["Gender"].value_counts()["Male"] / x["Gender"].count()\
        if "Male" in x["Gender"].value_counts() else 0
   
    return pd.Series(d)

%time
agg_df_nominal = data_demo.groupby("PostCode").apply(aggregate_group)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


In [37]:
def aggregate_ethnicity(x):
    d = {}
    d["Chinese Percentage"] = x["Ethnicity"].value_counts()["Chinese"] / x["Ethnicity"].count()\
        if "Chinese" in x["Ethnicity"].value_counts() else 0
    d["Indian Percentage"] = x["Ethnicity"].value_counts()["South Asian"] / x["Ethnicity"].count()\
        if "South Asian" in x["Ethnicity"].value_counts() else 0
    d["North American Percentage"] = x["Ethnicity"].value_counts()["North American"] / x["Ethnicity"].count()\
        if "North American" in x["Ethnicity"].value_counts() else 0
    return pd.Series(d)

agg_df_nominal_ethnicity = data_demo.groupby("PostCode").apply(aggregate_ethnicity)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [54]:
def aggregate_commute(x):
    d = {}
    d["Driver_percentage"] = x["Commute"].value_counts()["Car, truck, van - as a driver"] / x["Commute"].count()\
        if "Car, truck, van - as a driver" in x["Commute"].value_counts() else 0
    d["Passenger_percentage"] = x["Commute"].value_counts()["Car, truck, van - as a passenger"] / x["Commute"].count()\
        if "Car, truck, van - as a passenger" in x["Commute"].value_counts() else 0
    d["Public_transit_percentage"] = x["Commute"].value_counts()["Public transit"] / x["Commute"].count()\
        if "Public transit" in x["Commute"].value_counts() else 0
    d["Walk_percentage"] = x["Commute"].value_counts()["Walked"] / x["Commute"].count()\
        if "Walked" in x["Commute"].value_counts() else 0
    d["Bike_percentage"] = x["Commute"].value_counts()["Bicycle"] / x["Commute"].count()\
        if "Bicycle" in x["Commute"].value_counts() else 0
    return pd.Series(d)

agg_df_nominal_commute = data_demo.groupby("PostCode").apply(aggregate_commute)

In [42]:
def aggregate_education(x):
    d = {}
    d["No_degree_percentage"] = x["Education"].value_counts()["No certificate, diploma or degree"] / x["Education"].count()\
        if "No certificate, diploma or degree" in x["Education"].value_counts() else 0
    d["Secondary_school_percentage"] = x["Education"].value_counts()["Secondary school"] / x["Education"].count()\
        if "Secondary school" in x["Education"].value_counts() else 0
    d["Postsecondary"] = x["Education"].value_counts()["Postsecondary"] / x["Education"].count()\
        if "Postsecondary" in x["Education"].value_counts() else 0
    d["College_percentage"] = x["Education"].value_counts()["College"] / x["Education"].count()\
        if "College" in x["Education"].value_counts() else 0
    d["Bachelor_degree_percentage"] = x["Education"].value_counts()["Bachelor's degree"] / x["Education"].count()\
        if "Bachelor's degree" in x["Education"].value_counts() else 0
    d["Master_degree_percentage"] = x["Education"].value_counts()["Master's degree"] / x["Education"].count()\
        if "Master's degree" in x["Education"].value_counts() else 0
    d["Medical_degree_percentage"] = x["Education"].value_counts()["Degree in medicine & related"] / x["Education"].count()\
        if "Degree in medicine & related" in x["Education"].value_counts() else 0
    d["Doctorate_percentage"] = x["Education"].value_counts()["Doctorate"] / x["Education"].count()\
        if "Doctorate" in x["Education"].value_counts() else 0
    return pd.Series(d)

agg_df_nominal_education = data_demo.groupby("PostCode").apply(aggregate_education)

In [55]:
agg_df = agg_df_nominal_commute.merge(agg_df_nominal_education,on='PostCode') \
                               .merge(agg_df_nominal_ethnicity,on='PostCode')

In [57]:
agg_df.head()

,Driver_percentage,Passenger_percentage,Public_transit_percentage,Walk_percentage,Bike_percentage,No_degree_percentage,Secondary_school_percentage,Postsecondary,College_percentage,Bachelor_degree_percentage,Master_degree_percentage,Medical_degree_percentage,Doctorate_percentage,Chinese Percentage,Indian Percentage,North American Percentage
PostCode,,,,,,,,,,,,,,,,
A0A0B7,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.500000
A0A0C2,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.500000
A0A1A0,0.704918,0.049180,0.032787,0.016393,0.0,0.213115,0.295082,0.295082,0.163934,0.016393,0.016393,0.0,0.0,0.0,0.000000,0.442623
A0A1B0,0.787500,0.097917,0.037500,0.039583,0.0,0.018750,0.066667,0.535417,0.243750,0.104167,0.031250,0.0,0.0,0.0,0.000000,0.477083
A0A1C0,0.900092,0.059578,0.000000,0.011916,0.0,0.170486,0.151237,0.407883,0.120073,0.119157,0.031164,0.0,0.0,0.0,0.013749,0.410632


In [58]:
# Save the dataframe to a file to avoid re-processing.
agg_df_nominal.to_csv("data/demographics_2.csv", index=True)
agg_df.to_csv("data/demographics_4.csv", index=True)

In [10]:
agg_df_nominal.head()

,Immigrants Percentage,Owner Percentage,House Percentage,Male Percentage
PostCode,,,,
A0A0B7,0.000000,1.000000,1.000000,0.500000
A0A0C2,0.000000,1.000000,1.000000,0.500000
A0A1A0,0.049180,0.885246,1.000000,0.524590
A0A1B0,0.010417,0.795833,0.947917,0.497917
A0A1C0,0.009166,0.905591,0.963336,0.488543


## Preprocess and aggregate numeric data.

In [59]:
data_demo["Family Size"] = data_demo["Family Size"].str.split(" ", expand=True)[0]
data_demo["Family Size"] = pd.to_numeric(data_demo["Family Size"], errors="coerce")

In [60]:
data_demo.head()

,PostCode,Ethnicity,Immigration,Commute,Family Size,Dwelling,Ownership,Education,Income,Family Income,Age,Gender
0,E1V6V5,North American,Non-immigrants,"Car, truck, van - as a driver",1,Single-detached house,Owned,Postsecondary,34560,150658.0,25 to 29 years,Female
1,E1V6V5,British,Non-immigrants,"Car, truck, van - as a driver",2,Single-detached house,Owned,Postsecondary,57785,78950.0,45 to 49 years,Male
2,A0A0B7,British,Non-immigrants,"Car, truck, van - as a driver",1,Single-detached house,Owned,College,44055,111262.0,30 to 34 years,Female
3,A0A0B7,North American,Non-immigrants,"Car, truck, van - as a driver",2,Single-detached house,Owned,Postsecondary,35954,64925.0,35 to 39 years,Male
4,E4T0C1,North American,Non-immigrants,"Car, truck, van - as a driver",2,Single-detached house,Owned,Secondary school,59829,154926.0,60 to 64 years,Female


In [61]:
agg_df_numeric = data_demo.groupby("PostCode").agg(
    {"Family Size": np.median, 
     "Income": np.mean,
     "Family Income": np.mean,
    }
)

In [62]:
agg_df_numeric.head()

,Family Size,Income,Family Income
PostCode,,,
A0A0B7,1.5,40004.500000,88093.500000
A0A0C2,1.5,42080.000000,104456.000000
A0A1A0,1.0,40310.278689,93974.262295
A0A1B0,1.0,63745.333333,131847.975000
A0A1C0,1.0,55183.269478,123971.666361


## Preprocess and aggregate ordinal columns

In [63]:
ordered_age = {
    "Age": {
        '20 to 24 years': 1,
        '25 to 29 years': 2, 
        '30 to 34 years': 3, 
        '35 to 39 years': 4,
        '40 to 44 years': 5,
        '45 to 49 years': 6,
        '50 to 54 years': 7, 
        '55 to 59 years': 8,
        '60 to 64 years': 9,
        '65 years and over:': 10, 
    }
}
data_demo.replace(ordered_age, inplace=True)

In [64]:
data_demo.head()

,PostCode,Ethnicity,Immigration,Commute,Family Size,Dwelling,Ownership,Education,Income,Family Income,Age,Gender
0,E1V6V5,North American,Non-immigrants,"Car, truck, van - as a driver",1,Single-detached house,Owned,Postsecondary,34560,150658.0,2,Female
1,E1V6V5,British,Non-immigrants,"Car, truck, van - as a driver",2,Single-detached house,Owned,Postsecondary,57785,78950.0,6,Male
2,A0A0B7,British,Non-immigrants,"Car, truck, van - as a driver",1,Single-detached house,Owned,College,44055,111262.0,3,Female
3,A0A0B7,North American,Non-immigrants,"Car, truck, van - as a driver",2,Single-detached house,Owned,Postsecondary,35954,64925.0,4,Male
4,E4T0C1,North American,Non-immigrants,"Car, truck, van - as a driver",2,Single-detached house,Owned,Secondary school,59829,154926.0,9,Female


In [65]:
data_demo["Age"] = pd.to_numeric(data_demo["Age"], errors="coerce")
agg_df_ordinal = data_demo.groupby("PostCode").agg(
    {"Age": np.mean}
)

In [66]:
agg_df_ordinal.head()

,Age
PostCode,
A0A0B7,3.500000
A0A0C2,7.000000
A0A1A0,6.615385
A0A1B0,5.632597
A0A1C0,5.065398


## Join all dataframes

In [67]:
agg_df_nominal = pd.read_csv("data/demographics_2.csv")

In [68]:
agg_df = agg_df.merge(agg_df_numeric, on='PostCode') \
               .merge(agg_df_ordinal, on='PostCode') \
               .merge(agg_df_nominal, on="PostCode")

In [69]:
agg_df.head()

,PostCode,Driver_percentage,Passenger_percentage,Public_transit_percentage,Walk_percentage,Bike_percentage,No_degree_percentage,Secondary_school_percentage,Postsecondary,College_percentage,...,Indian Percentage,North American Percentage,Family Size,Income,Family Income,Age,Immigrants Percentage,Owner Percentage,House Percentage,Male Percentage
0,A0A0B7,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.500000,...,0.000000,0.500000,1.5,40004.500000,88093.500000,3.500000,0.000000,1.000000,1.000000,0.500000
1,A0A0C2,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,...,0.000000,0.500000,1.5,42080.000000,104456.000000,7.000000,0.000000,1.000000,1.000000,0.500000
2,A0A1A0,0.704918,0.049180,0.032787,0.016393,0.0,0.213115,0.295082,0.295082,0.163934,...,0.000000,0.442623,1.0,40310.278689,93974.262295,6.615385,0.049180,0.885246,1.000000,0.524590
3,A0A1B0,0.787500,0.097917,0.037500,0.039583,0.0,0.018750,0.066667,0.535417,0.243750,...,0.000000,0.477083,1.0,63745.333333,131847.975000,5.632597,0.010417,0.795833,0.947917,0.497917
4,A0A1C0,0.900092,0.059578,0.000000,0.011916,0.0,0.170486,0.151237,0.407883,0.120073,...,0.013749,0.410632,1.0,55183.269478,123971.666361,5.065398,0.009166,0.905591,0.963336,0.488543


In [70]:
agg_df.to_csv("data/demographics_5.csv", index=False)